# Setup

The main focus of this notebook is to analyze and mitigate potential biases in a dataset. The notebook uses various techniques and technologies, such as [AIF360](https://aif360.mybluemix.net), [Fairlearn](https://fairlearn.org) , and [scikit-learn](https://scikit-learn.org/stable/), to preprocess and analyze the data, and to train and evaluate machine learning models. The notebook also includes visualizations and statistics to help understand the distribution and correlations of the data, and to identify any potential biases.

**PLEASE NOTE**: The notebook must be configured with a dataset and some configuration variables.  
The ***protected attributes*** must be categorical and binary (0,1), but the original column must be mantained as a continuous variable.

## Colab
In order to run this notebook in Google Colab, you have to:
1. Upload the project folder to your Google Drive
2. Mount your Google Drive in the next code cell
3. Update the `path_to_project` variable, with the path to the project folder in your Google Drive (e.g. `path_to_project = '/content/drive/MyDrive/Colab/project'`), this can be seen from the file tab on the left of the Colab interface
4. Update the pip installation command  (e.g. `!pip install -r /content/drive/MyDrive/Colab/project/requirements.txt`)

In [1]:
try:
  from google.colab import drive
  drive.mount('/content/drive')
  import sys
  path_to_project = '/content/drive/MyDrive/FairAlgorithm'  #UPDATE THIS LINE
  #path_to_project = '/content/drive/MyDrive/Polimi/Last Semester/FairAlgorithm'
  sys.path.append(path_to_project)
  !sudo apt install libcairo2-dev pkg-config python3-dev
  %pip install -r /content/drive/MyDrive/FairAlgorithm/source/requirements.txt  #UPDATE THIS LINE
  #%pip install -r '/content/drive/MyDrive/Polimi/Last Semester/FairAlgorithm/source/requirements.txt'  #UPDATE THIS LINE
  IN_COLAB = True
except:
  IN_COLAB = False

Mounted at /content/drive
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-dev is already the newest version (3.10.6-1~22.04.1).
python3-dev set to manually installed.
The following packages were automatically installed and are no longer required:
  libbz2-dev libpkgconf3 libreadline-dev
Use 'sudo apt autoremove' to remove them.
The following additional packages will be installed:
  libblkid-dev libblkid1 libcairo-script-interpreter2 libffi-dev
  libglib2.0-dev libglib2.0-dev-bin libice-dev liblzo2-2 libmount-dev
  libmount1 libpixman-1-dev libselinux1-dev libsepol-dev libsm-dev
  libxcb-render0-dev libxcb-shm0-dev
Suggested packages:
  libcairo2-doc libgirepository1.0-dev libglib2.0-doc libgdk-pixbuf2.0-bin
  | libgdk-pixbuf2.0-dev libxml2-utils libice-doc cryptsetup-bin libsm-doc
The following packages will be REMOVED:
  pkgconf r-base-dev
The following NEW packages will be installed:
  libblkid-dev libcairo-script-interpreter2 

In [2]:
import numpy as np
import pandas as pd
np.random.seed(1234)
from rich import print
from rich.columns import Columns
from rich.panel import Panel
from rich.align import Align
from source.utils.print_util import *
from source.utils.data_preprocessing import *
import matplotlib.pyplot as plt
from fairlearn.metrics import  MetricFrame, count, false_negative_rate, false_positive_rate, selection_rate, equalized_odds_difference, demographic_parity_difference
from fairlearn.preprocessing import CorrelationRemover
from fairlearn.adversarial import AdversarialFairnessClassifier
from fairlearn.reductions import ExponentiatedGradient, Moment
from fairlearn.postprocessing import ThresholdOptimizer
from aif360.datasets import BinaryLabelDataset, StructuredDataset, StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric, Metric
from aif360.algorithms.preprocessing import DisparateImpactRemover, Reweighing, LFR, OptimPreproc
from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools
from aif360.algorithms.inprocessing import PrejudiceRemover, AdversarialDebiasing, ExponentiatedGradientReduction
from aif360.algorithms.postprocessing import RejectOptionClassification, CalibratedEqOddsPostprocessing, EqOddsPostprocessing
from aif360.algorithms import Transformer
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

from tqdm.notebook import tqdm
import pickle

pip install 'aif360[inFairness]'


In [3]:
from sklearn.model_selection import cross_validate, cross_val_score, cross_val_predict, train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, recall_score, accuracy_score, precision_score, confusion_matrix, roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, normalize
from source.utils.config import *

In [4]:
!pip install BlackBoxAuditing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 28.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for BlackBoxAuditing: filename=BlackBoxAuditing-0.1.54-py2.py3-none-any.whl size=1394756 sha256=0ca7ce0c85ceee2d2c6cf43e763acb56f1c20adac38d93d4e0683d225251ba5b
  Stored in directory: /root/.cache/pip/wheels/c0/4f/b1/80e1b0790df07536470758fe0a4f9ff8fa942fd9fe30bbb192
Successfully built BlackBoxAuditing


## Configure the notebook


In the next code cell, set all the variables that are used throughout the notebook.  
The variables are used to configure the notebook, and to set the paths to the data files.

Modify the variables in the next code cell to configure the notebook

- `dataset_name`: The name of the dataset file.
- `sensible_attribute`: The sensible attribute to use for bias mitigation.

In [5]:
dataset_name = "diabetes-predictionOLD"
#dataset_name2 = "diabetes-prediction"

# from source.utils.config import *
# print(datasets_config)

# dataset_config = datasets_config[dataset_name2]
# ignore_cols = dataset_config['ignore_cols']
# target_variable = dataset_config['target_variable']
# target_variable_labels = dataset_config['target_variable_labels']
# sensible_attributes = dataset_config['sensible_attributes']
# sensible_attribute = sensible_attributes[0]
# default_mappings = dataset_config.get('default_mappings', {})

dataset_path = path_to_project + '/data/preprocessed/preprocessed-{}.csv'.format(dataset_name) if IN_COLAB else 'data/preprocessed/preprocessed-{}.csv'.format(dataset_name)
df = pd.read_csv(dataset_path)
#df = df.drop(columns=ignore_cols)
feature_cols = df.columns

In [6]:
df.head(5)

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,race_category,gender_category,...,glyburide-metformin_category,glipizide-metformin_category,glimepiride-pioglitazone_category,metformin-rosiglitazone_category,metformin-pioglitazone_category,age_mean,age_category,change_category,diabetes-medication_category,diabetes
0,2,49,0,19,0,0,1,9,1,0,...,0,0,0,0,0,55,1,1,1,0
1,2,19,0,15,0,0,2,8,1,0,...,0,0,0,0,0,85,1,0,0,1
2,1,52,1,12,0,0,0,9,0,0,...,0,0,0,0,0,65,1,0,1,0
3,7,47,1,14,0,0,1,6,1,1,...,0,0,0,0,0,65,1,0,1,1
4,7,41,2,32,0,0,0,9,1,0,...,0,0,0,0,0,85,1,1,1,0


In [ ]:
#INPUT
dataset_name = "diabetes-prediction"

In [ ]:
# if dataset_name == "diabetes-women":
#   ignore_cols = ['Age']
#   target_variable = 'Outcome'
#   target_variable_labels= ['No Diabetes','Diabetes']
#   sensible_attribute = 'AgeCategory'

#   default_mappings = {
#       'label_maps': [{1.0: 'Diabetic', 0.0: 'NonDiabetic'}],
#       'protected_attribute_maps': [{1.0: 'Adult', 0.0: 'Young'}]
#   }

# elif dataset_name == "diabetes-prediction":
#   ignore_cols = []
#   target_variable = 'diabetes'
#   target_variable_labels= ['No Diabetes','Diabetes']
#   sensible_attribute = 'race_category'

#   default_mappings = {
#       'label_maps': [{1.0: 'Diabetic', 0.0: 'NonDiabetic'}],
#       'protected_attribute_maps': [{1.0: 'Caucasian', 0.0: 'Non-Caucasian'}]
#   }

from source.utils.config import *
print(datasets_config)

dataset_config = datasets_config[dataset_name]
ignore_cols = dataset_config['ignore_cols']
target_variable = dataset_config['target_variable']
target_variable_labels = dataset_config['target_variable_labels']
sensible_attributes = dataset_config['sensible_attributes']
sensible_attribute = sensible_attributes[0]
default_mappings = dataset_config.get('default_mappings', {})

dataset_path = path_to_project + '/data/preprocessed/preprocessed-{}.csv'.format(dataset_name) if IN_COLAB else 'data/preprocessed/preprocessed-{}.csv'.format(dataset_name)
df = pd.read_csv(dataset_path)
df = df.drop(columns=ignore_cols)
feature_cols = df.columns

{
    'diabetes-women': {
        'ignore_cols': ['Age'],
        'target_variable': 'Outcome',
        'target_variable_labels': [1, 0],
        'sensible_attributes': ['AgeCategory'],
        'default_mappings': {
            'label_maps': [{1.0: 'Diabetic', 0.0: 'NonDiabetic'}],
            'protected_attribute_maps': [{1.0: 'Adult', 0.0: 'Young'}]
        }
    },
    'sepsis': {
        'ignore_cols': [],
        'target_variable': 'Mortality',
        'target_variable_labels': [1, 0],
        'sensible_attributes': ['Gender_cat', 'Age_cat'],
        'default_mappings': {}
    },
    'aids': {
        'ignore_cols': [],
        'target_variable': 'cid',
        'target_variable_labels': [1, 0],
        'sensible_attributes': ['homo_cat', 'race_cat', 'age_cat'],
        'default_mappings': {}
    },
    'myocardial-infarction': {
        'ignore_cols': [],
        'target_variable': 'LET_IS_cat',
        'target_variable_labels': [1, 0],
        'sensible_attributes': ['SEX'],
        'default_mappings': {
            'label_maps': [{1: 'Complication', 0: 'No Alzheimer'}],
            'protected_attribute_maps': [{0: 'Female', 1: 'Male'}]
        }
    },
    'alzheimer-disease': {
        'ignore_cols': [],
        'target_variable': 'Diagnosis',
        'target_variable_labels': [1, 0],
        'sensible_attributes': ['Ethnicity_cat', 'Gender_cat'],
        'default_mappings': {
            'label_maps': [{1: 'Alzheimer', 0: 'No Alzheimer'}],
            'protected_attribute_maps': [
                {'Ethnicity_cat': {1: 'Caucasian', 0: 'Non-Caucasian'}, 'Gender_cat': {1: 'Male', 0: 'Female'}}
            ]
        }
    },
    'diabetes-prediction': {
        'ignore_cols': [],
        'target_variable': 'diabetes',
        'target_variable_labels': [1, 0],
        'sensible_attributes': ['race_category'],
        'default_mappings': {
            'label_maps': [{1.0: 'Diabetic', 0.0: 'NonDiabetic'}],
            'protected_attribute_maps': [{1.0: 'Caucasian', 0.0: 'Non-Caucasian'}]
        }
    },
    'stroke-prediction': {
        'ignore_cols': [],
        'target_variable': 'stroke_prediction',
        'target_variable_labels': [1, 0],
        'sensible_attributes': ['residence_category'],
        'default_mappings': {
            'label_maps': [{1.0: 'Stroke', 0.0: 'No Stroke'}],
            'protected_attribute_maps': [{1.0: 'Urban', 0.0: 'Rural'}]
        }
    }
}

In [ ]:
from sklearn import preprocessing

# Build a list with format (feature, correlation with target)
features_corr = [(column, correlation) for column, correlation in zip(df.columns, df.corr()[target_variable])] # Modify stroke with the target_variable

# Sort the features by correlation
sorted_features = sorted(features_corr, key=lambda x: x[1], reverse=True)

# Clean and take top 4
main_features = [feature[0] for feature in sorted_features if feature[1] > 0][:5]

# Add sensitive attribute
main_features = main_features + [sensible_attribute]

# Create a new reduced df
df_reduced = df[main_features]
df_reduced = df_reduced[:5000]

In [ ]:
apply_dimensionality_reduction = True
config = {}
config['df']= df
config['target_variable'] = target_variable
config['sensible_attribute'] = sensible_attribute
config['path_to_project'] = path_to_project
config['n_splits'] = n_splits
config['models'] = models
config['n_estimators'] = n_estimators
config['random_seed'] = random_seed

In [ ]:
def unpack_config(config):
  return config['df'], config['target_variable'], config['sensible_attribute'], config['path_to_project'], config['n_splits'], config['models'], config['n_estimators'],config['random_seed']

## Load the data
Data is loaded from the file specified in the `dataset_path` variable using the `pandas` library.  
[pandas](https://pandas.pydata.org) is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool,
built on top of the Python programming language.

In [ ]:
df.head(5)

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,race_category,gender_category,...,glyburide-metformin_category,glipizide-metformin_category,glimepiride-pioglitazone_category,metformin-rosiglitazone_category,metformin-pioglitazone_category,age_mean,age_category,change_category,diabetes-medication_category,diabetes
0,1,41,0,1,0,0,0,1,1,0,...,0,0,0,0,0,5,0,0,0,0
1,3,59,0,18,0,0,0,9,1,0,...,0,0,0,0,0,15,0,1,1,1
2,2,44,1,16,0,0,0,7,1,1,...,0,0,0,0,0,35,1,1,1,0
3,1,51,0,8,0,0,0,5,1,1,...,0,0,0,0,0,45,1,1,1,0
4,3,31,6,16,0,0,0,9,1,1,...,0,0,0,0,0,55,1,0,1,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73450 entries, 0 to 73449
Data columns (total 38 columns):
 #   Column                             Non-Null Count  Dtype
---  ------                             --------------  -----
 0   time_in_hospital                   73450 non-null  int64
 1   num_lab_procedures                 73450 non-null  int64
 2   num_procedures                     73450 non-null  int64
 3   num_medications                    73450 non-null  int64
 4   number_outpatient                  73450 non-null  int64
 5   number_emergency                   73450 non-null  int64
 6   number_inpatient                   73450 non-null  int64
 7   number_diagnoses                   73450 non-null  int64
 8   race_category                      73450 non-null  int64
 9   gender_category                    73450 non-null  int64
 10  metformin_category                 73450 non-null  int64
 11  repaglinide_category               73450 non-null  int64
 12  nateglinide_catego

## Mitigation utils


In [ ]:
def train_test_splitting(df, n_splits):
  df_splitting = {}

  w = int(len(df)/n_splits)
  window = w
  start_point = 0
  for i in range(0,n_splits):
      train = {}
      test = {}
      df_train_1 = {}
      df_train_2 = {}
      df_test = df[start_point:window]
      if i != 0:
        df_train_1 = df[0: start_point]

      if i != n_splits-1:
        df_train_2 = df[window: len(df)]

      if (i != 0 and  i != n_splits-1):
        concat_df = [df_train_1, df_train_2]
        df_train = pd.concat(concat_df)
      elif i != 0:
        df_train = df_train_1
      else:
        df_train = df_train_2

      start_point= window
      window = window + w

      df_splitting[i] = {'train': df_train, 'test': df_test}
  return df_splitting

In [ ]:
def df_X_Y_split(df_train, df_test, target_variable):
  Y_train = df_train[target_variable]
  X_train = df_train.drop(target_variable, axis=1)
  Y_test = df_test[target_variable]
  X_test = df_test.drop(target_variable, axis=1)
  return X_train, Y_train, X_test, Y_test

In [ ]:
def df_X_Y_split_2D(df_train, df_test, target_variable, sensible_attribute):
  Y_train = df_train[target_variable].values
  S_train = df_train[sensible_attribute].values
  X_train = df_train.drop(target_variable, axis=1)
  X_train = X_train.drop(sensible_attribute, axis=1)

  Y_test = df_test[target_variable]
  X_test = df_test.drop(target_variable, axis=1)
  S_test = df_test[sensible_attribute].values

  X_train = normalize(X_train)
  X_test = normalize(X_test)

  pca = PCA(n_components=2)
  X_train2D = pca.fit(X_train).transform(X_train)
  X_test2D = pca.fit(X_test).transform(X_test)
  return X_train2D, Y_train, X_test2D, Y_test, S_train, S_test

In [ ]:
def df_X_Y_split_S(df_train, df_test, target_variable, sensible_attribute):
  Y_train = df_train[target_variable].values
  S_train = df_train[sensible_attribute].values
  X_train = df_train.drop(target_variable, axis=1)

  Y_test = df_test[target_variable]
  X_test = df_test.drop(target_variable, axis=1)
  S_test = df_test[sensible_attribute].values

  return X_train, Y_train, X_test, Y_test, S_train, S_test

In [ ]:
def compute_predictions_and_tests(df, target_variable, n_splits, models, n_estimators, random_seed):
  predicted_and_real_values = {}
  for model_name in tqdm(models):
    clf = models[model_name]
    df_splitting = train_test_splitting(df, n_splits)
    pred_and_y = {}
    for i in range(0,n_splits):
      df_split = df_splitting[i]
      df_train = df_split['train']
      df_test = df_split['test']

      X_train, Y_train, X_test, Y_test = df_X_Y_split(df_train, df_test, target_variable)
      clf.fit(X_train,Y_train)
      y_pred = clf.predict(X_test)

      S_test = X_test[sensible_attribute].values
      pred_and_y[i] = {'y_test': Y_test.to_numpy().astype(int), 'y_pred': y_pred.astype(int), 's_test':  S_test.astype(int)}

    predicted_and_real_values[model_name] = pred_and_y

  return predicted_and_real_values


In [ ]:
def fl_am_compute_predictions_and_tests(df, target_variable, sensible_attribute, n_splits, classifier):
  predicted_and_real_values = {}

  df_splitting = train_test_splitting(df, n_splits)
  pred_and_y = {}
  for i in range(0,n_splits):

    df_split = df_splitting[i]
    df_train = df_split['train']
    df_test = df_split['test']
    ##Input data should be in 2-dimensions
    X_train2D, Y_train, X_test2D, Y_test, S_train, S_test = df_X_Y_split_2D(df_train, df_test, target_variable, sensible_attribute)
    #print(len(X_train2D), len(Y_train),len(S_train))

    #classifier is given in in-processing mitigation
    predictor = classifier.fit(X_train2D, Y_train, sensitive_features=S_train)
    y_pred = predictor.predict(X_test2D)
    pred_and_y[i] = {'y_test': Y_test.to_numpy(), 'y_pred': y_pred, 's_test':  S_test }

  predicted_and_real_values = pred_and_y

  return predicted_and_real_values

In [ ]:
def fl_to_compute_predictions_and_tests(df, target_variable, sensible_attribute, n_splits, classifier):
  predicted_and_real_values = {}

  df_splitting = train_test_splitting(df, n_splits)
  pred_and_y = {}
  for i in range(0,n_splits):

    df_split = df_splitting[i]
    df_train = df_split['train']
    df_test = df_split['test']
    ##Input data should be in 2-dimensions
    X_train, Y_train, X_test, Y_test, S_train, S_test = df_X_Y_split_S(df_train, df_test, target_variable, sensible_attribute)
    #print(len(X_train2D), len(Y_train),len(S_train))

    #classifier is given in in-processing mitigation
    predictor = classifier.fit(X_train, Y_train, sensitive_features=S_train)
    y_pred = predictor.predict(X_test, sensitive_features=S_test)
    pred_and_y[i] = {'y_test': Y_test.to_numpy(), 'y_pred': y_pred, 's_test':  S_test}

  predicted_and_real_values = pred_and_y

  return predicted_and_real_values

In [ ]:
def save_predictions_and_tests(predicted_and_test, dataset_name, sensible_attribute, mitigation,path_to_project):
  save_path = path_to_project + '/data/predictions_and_tests/pred_test-{}-{}-{}.p'.format(dataset_name, sensible_attribute, mitigation)
  with open(save_path, 'wb') as fp:
    pickle.dump(predicted_and_test, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
def save_mitigated_dataset(mitigated_dataset, path_to_project,dataset_name, sensible_attribute, mitigation):
  mitigated_dataset.to_csv("{}/data/mitigated/mitigated-{}-{}-{}.csv".format(path_to_project,dataset_name, sensible_attribute, mitigation), sep=',', index=False, encoding='utf-8')

In [ ]:
def check_results(results):
  for model_name in tqdm(models):
    for i in range(0,n_splits):
      y_test = results[model_name][i]['y_test']
      y_pred = results[model_name][i]['y_pred']
      for j in range(0,67):
        if y_test[j] != y_pred[j]:
          print(y_test[j], y_pred[j])

## Mitigation utils OLD


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn import metrics

In [ ]:
all_mitigations = ['original','fl-cr', 'fl-to', 'aif360-rw', 'aif360-di', 'aif360-lfr', 'aif360-op', 'aif360-ad', 'aif360-pr', 'aif360-er', 'aif360-ce', 'aif360-eo', 'aif360-roc']
mitigation = 'aif360-roc'
without_model_mitigations = ['aif360-ad', 'aif360-pr', 'aif360-er']
new_dataset_mitigations = ["fl-cr", "aif360-di", "aif360-op" "aif360-lfr"]

In [ ]:
def compute_mean_std_dev(metric_list, models):
  metric_dict = {}
  if models is not None:
    for model_name in (models):
      metric = np.array(metric_list[model_name])
      mean_metric = metric.mean()
      std_metric = metric.std()
      metric_dict[model_name] = [mean_metric, std_metric]
  else:
    metric = np.array(metric_list)
    mean_metric = metric.mean()
    std_metric = metric.std()
    metric_dict = [mean_metric, std_metric]
  return metric_dict

In [ ]:
def compute_performance_metrics(predictions_and_tests, models, n_splits):
  accuracy, precision, recall, f1_score = compute_scores(predictions_and_tests, models, n_splits)

  if mitigation not in without_model_mitigations:
    #for each model compute mean and standard deviation
    acc = compute_mean_std_dev(accuracy, models)
    prec = compute_mean_std_dev(precision, models)
    rec = compute_mean_std_dev(recall, models)
    f1 = compute_mean_std_dev(f1_score, models)
  else:
    acc = compute_mean_std_dev(accuracy, None)
    prec = compute_mean_std_dev(precision, None)
    rec = compute_mean_std_dev(recall, None)
    f1 = compute_mean_std_dev(f1_score, None)

  performance_metrics = {}
  performance_metrics['accuracy'] = acc
  performance_metrics['precision'] = prec
  performance_metrics['recall'] = rec
  performance_metrics['f1_score'] = f1

  return performance_metrics

In [ ]:
def compute_scores(predictions_and_tests, models, n_splits):
  precision = {}
  recall = {}
  accuracy = {}
  f1_score = {}

  if mitigation not in without_model_mitigations:
    for model_name in (models):

      precisions = []
      recalls = []
      accuracys = []
      f1_scores = []
      for i in range(0,n_splits):
        y_test = predictions_and_tests[model_name][i]['y_test']
        y_pred = predictions_and_tests[model_name][i]['y_pred']
        #print(len(y_test), len(y_pred))
        precisions.append(metrics.precision_score(y_test, y_pred))
        recalls.append(metrics.recall_score(y_test, y_pred))
        accuracys.append(metrics.accuracy_score(y_test, y_pred))
        f1_scores.append(metrics.f1_score(y_test, y_pred))
      precision[model_name] = precisions
      recall[model_name] = recalls
      accuracy[model_name] = accuracys
      f1_score[model_name] = f1_scores
  else:
    precisions = []
    recalls = []
    accuracys = []
    f1_scores = []
    for i in range(0,n_splits):
        y_test = predictions_and_tests[i]['y_test']
        y_pred = predictions_and_tests[i]['y_pred']
        #print(len(y_test), len(y_pred))
        precisions.append(metrics.precision_score(y_test, y_pred))
        recalls.append(metrics.recall_score(y_test, y_pred))
        accuracys.append(metrics.accuracy_score(y_test, y_pred))
        f1_scores.append(metrics.f1_score(y_test, y_pred))
    precision = precisions
    recall = recalls
    accuracy = accuracys
    f1_score = f1_scores
  return accuracy, precision, recall, f1_score

In [ ]:
def train_test_splitting(df, n_splits):
  df_splitting = {}

  w = int(len(df)/n_splits)
  window = w
  start_point = 0
  for i in range(0,n_splits):
      train = {}
      test = {}
      df_train_1 = {}
      df_train_2 = {}
      df_test = df[start_point:window]
      if i != 0:
        df_train_1 = df[0: start_point]

      if i != n_splits-1:
        df_train_2 = df[window: len(df)]

      if (i != 0 and  i != n_splits-1):
        concat_df = [df_train_1, df_train_2]
        df_train = pd.concat(concat_df)
      elif i != 0:
        df_train = df_train_1
      else:
        df_train = df_train_2

      start_point= window
      window = window + w

      df_splitting[i] = {'train': df_train, 'test': df_test}
  return df_splitting

In [ ]:
#print(train_test_splitting(df, n_splits))

In [ ]:
def df_X_Y_split(df_train, df_test, target_variable):
  Y_train = df_train[target_variable]
  X_train = df_train.drop(target_variable, axis=1)
  Y_test = df_test[target_variable]
  X_test = df_test.drop(target_variable, axis=1)
  return X_train, Y_train, X_test, Y_test

In [ ]:
def df_X_Y_split_2D(df_train, df_test, target_variable, sensible_attribute):
  Y_train = df_train[target_variable].values
  S_train = df_train[sensible_attribute].values
  X_train = df_train.drop(target_variable, axis=1)
  X_train = X_train.drop(sensible_attribute, axis=1)

  Y_test = df_test[target_variable]
  X_test = df_test.drop(target_variable, axis=1)
  S_test = df_test[sensible_attribute].values

  X_train = normalize(X_train)
  X_test = normalize(X_test)

  pca = PCA(n_components=2)
  X_train2D = pca.fit(X_train).transform(X_train)
  X_test2D = pca.fit(X_test).transform(X_test)
  return X_train2D, Y_train, X_test2D, Y_test, S_train, S_test

In [ ]:
def df_X_Y_split_S(df_train, df_test, target_variable, sensible_attribute):
  Y_train = df_train[target_variable].values
  S_train = df_train[sensible_attribute].values
  X_train = df_train.drop(target_variable, axis=1)

  Y_test = df_test[target_variable]
  X_test = df_test.drop(target_variable, axis=1)
  S_test = df_test[sensible_attribute].values

  return X_train, Y_train, X_test, Y_test, S_train, S_test

In [ ]:
def compute_predictions_and_tests(df, target_variable, n_splits, models, n_estimators, random_seed):
  predicted_and_real_values = {}
  for model_name in tqdm(models):
    clf = models[model_name]
    df_splitting = train_test_splitting(df, n_splits)
    pred_and_y = {}
    for i in range(0,n_splits):
      df_split = df_splitting[i]
      df_train = df_split['train']
      df_test = df_split['test']

      X_train, Y_train, X_test, Y_test = df_X_Y_split(df_train, df_test, target_variable)
      clf.fit(X_train,Y_train)
      y_pred = clf.predict(X_test)

      S_test = X_test[sensible_attribute].values
      pred_and_y[i] = {'y_test': Y_test.to_numpy().astype(int), 'y_pred': y_pred.astype(int), 's_test':  S_test.astype(int)}

    predicted_and_real_values[model_name] = pred_and_y

  return predicted_and_real_values


In [ ]:
def fl_am_compute_predictions_and_tests(df, target_variable, sensible_attribute, n_splits, classifier):
  predicted_and_real_values = {}

  df_splitting = train_test_splitting(df, n_splits)
  pred_and_y = {}
  for i in range(0,n_splits):

    df_split = df_splitting[i]
    df_train = df_split['train']
    df_test = df_split['test']
    ##Input data should be in 2-dimensions
    X_train2D, Y_train, X_test2D, Y_test, S_train, S_test = df_X_Y_split_2D(df_train, df_test, target_variable, sensible_attribute)
    #print(len(X_train2D), len(Y_train),len(S_train))

    #classifier is given in in-processing mitigation
    predictor = classifier.fit(X_train2D, Y_train, sensitive_features=S_train)
    y_pred = predictor.predict(X_test2D)
    pred_and_y[i] = {'y_test': Y_test.to_numpy(), 'y_pred': y_pred, 's_test':  S_test }

  predicted_and_real_values = pred_and_y

  return predicted_and_real_values

In [ ]:
def fl_to_compute_predictions_and_tests(df, target_variable, sensible_attribute, n_splits, classifier):
  predicted_and_real_values = {}

  df_splitting = train_test_splitting(df, n_splits)
  pred_and_y = {}
  for i in range(0,n_splits):

    df_split = df_splitting[i]
    df_train = df_split['train']
    df_test = df_split['test']
    ##Input data should be in 2-dimensions
    X_train, Y_train, X_test, Y_test, S_train, S_test = df_X_Y_split_S(df_train, df_test, target_variable, sensible_attribute)
    #print(len(X_train2D), len(Y_train),len(S_train))

    #classifier is given in in-processing mitigation
    predictor = classifier.fit(X_train, Y_train, sensitive_features=S_train)
    y_pred = predictor.predict(X_test, sensitive_features=S_test)
    pred_and_y[i] = {'y_test': Y_test.to_numpy(), 'y_pred': y_pred, 's_test':  S_test}

  predicted_and_real_values = pred_and_y

  return predicted_and_real_values

In [ ]:
def save_predictions_and_tests(predicted_and_test, mitigation, dataset_name, path_to_project):
  save_path = path_to_project + '/data/predictions_and_tests/pred_test-{}-{}.p'.format(dataset_name, mitigation)
  with open(save_path, 'wb') as fp:
    pickle.dump(predicted_and_test, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
def save_mitigated_dataset(mitigated_dataset, path_to_project,dataset_name, mitigation):
  mitigated_dataset.to_csv("{}/data/mitigated/mitigated-{}-{}.csv".format(path_to_project,dataset_name, mitigation), sep=',', index=False, encoding='utf-8')

In [ ]:
def check_results(results):
  for model_name in tqdm(models):
    for i in range(0,n_splits):
      y_test = results[model_name][i]['y_test']
      y_pred = results[model_name][i]['y_pred']
      for j in range(0,67):
        if y_test[j] != y_pred[j]:
          print(y_test[j], y_pred[j])

# Fairlearn
[Fairlearn](https://fairlearn.org) is an open source toolkit that empowers developers of artificial intelligence (AI) systems to assess their system's fairness and mitigate any observed unfairness issues.

### Correlation Remover
Pre-processing: mitigated dataset changes

MITIGATED DATASET

ALPHA PARAMETER SET TO DEFAULT

NO MODEL ENFORCEMENT

In [ ]:
def fl_cr(config):
  mitigation='fl-cr'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)
  X_raw = df.drop(target_variable, axis=1)
  X_raw = pd.get_dummies(X_raw)
  y = df[target_variable].values

  cr = CorrelationRemover(sensitive_feature_ids=[sensible_attribute])
  X_cr = cr.fit_transform(X_raw)

  X_cr = pd.DataFrame(
      X_cr, columns = X_raw.drop(sensible_attribute, axis=1).columns
  )

  X_cr[sensible_attribute] = X_raw[sensible_attribute]
  mit_fl_cr = X_cr.copy(deep=True)
  mit_fl_cr[target_variable] = y

  save_mitigated_dataset(mit_fl_cr,path_to_project,dataset_name, sensible_attribute, mitigation)
  predictions_and_tests = compute_predictions_and_tests(mit_fl_cr, target_variable, n_splits, models, n_estimators, random_seed)
  save_predictions_and_tests(predictions_and_tests, dataset_name, sensible_attribute, mitigation,path_to_project)
  return mit_fl_cr, predictions_and_tests

In [ ]:
print(fl_cr(config))

  0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

(
           time_in_hospital  num_lab_procedures  num_procedures  num_medications  \
0              1.021027           41.328073       -0.036802         0.756052   
1              3.021027           59.328073       -0.036802        17.756052   
2              2.021027           44.328073        0.963198        15.756052   
3              1.021027           51.328073       -0.036802         7.756052   
4              3.021027           31.328073        5.963198        15.756052   
...                 ...                 ...             ...              ...   
73445          5.021027           21.328073        0.963198        15.756052   
73446          2.934578           49.979236        0.114506        16.759017   
73447          4.934578           31.979236        3.114506        18.759017   
73448         10.021027           45.328073        1.963198        20.756052   
73449          6.021027           13.328073        2.963198         2.756052   

       number_outpatient  number_emergency  number_inpatient  \
0                    0.0               0.0          0.004734   
1                    0.0               0.0          0.004734   
2                    0.0               0.0          0.004734   
3                    0.0               0.0          0.004734   
4                    0.0               0.0          0.004734   
...                  ...               ...               ...   
73445                0.0               0.0          1.004734   
73446                0.0               0.0         -0.014728   
73447                0.0               0.0          0.985272   
73448                0.0               0.0          1.004734   
73449                0.0               0.0          0.004734   

       number_diagnoses  gender_category  metformin_category  ...  \
0              0.889340        -0.017107           -0.001078  ...   
1              8.889340        -0.017107           -0.001078  ...   
2              6.889340         0.982893           -0.001078  ...   
3              4.889340         0.982893           -0.001078  ...   
4              8.889340         0.982893           -0.001078  ...   
...                 ...              ...                 ...  ...   
73445          8.889340        -0.017107           -0.001078  ...   
73446          9.344308         1.053227            1.003355  ...   
73447          9.344308         0.053227            0.003355  ...   
73448          8.889340        -0.017107           -0.001078  ...   
73449          8.889340         0.982893           -0.001078  ...   

       glipizide-metformin_category  glimepiride-pioglitazone_category  \
0                         -0.000026                          -0.000004   
1                         -0.000026                          -0.000004   
2                         -0.000026                          -0.000004   
3                         -0.000026                          -0.000004   
4                         -0.000026                          -0.000004   
...                             ...                                ...   
73445                     -0.000026                          -0.000004   
73446                      0.000082                           0.000014   
73447                      0.000082                           0.000014   
73448                     -0.000026                          -0.000004   
73449                     -0.000026                          -0.000004   

       metformin-rosiglitazone_category  metformin-pioglitazone_category  \
0                                   0.0                        -0.000004   
1                                   0.0                        -0.000004   
2                                   0.0                        -0.000004   
3                                   0.0                        -0.000004   
4                                   0.0                        -0.000004   
...                                 ...                              ...   
73445         

### Threshold Optimizer
Post-processing: change predictions, mitigated dataset is equal to the original one

SET THE MODEL TO CLASSIFIER - NO MODEL ENFORCEMENT

NO PARAMETERS

NO MITIGATED DATASET

In [ ]:
def fl_to(config):
  mitigation='fl-to'
  df, target_variable, sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)
  predictions_and_tests= {}

  for model_name in tqdm(models):
    classifier =  models[model_name]
    TO = ThresholdOptimizer(estimator=classifier)
    pred_and_y = fl_to_compute_predictions_and_tests(df, target_variable, sensible_attribute, n_splits, TO)
    predictions_and_tests[model_name] = pred_and_y

  save_predictions_and_tests(predictions_and_tests, dataset_name, sensible_attribute, mitigation, path_to_project)
  return predictions_and_tests

In [ ]:
print(fl_to(config))

  0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/fairlearn/postprocessing/_threshold_optimizer.py:285: FutureWarning: 'predict_method' default value is changed from 'predict' to 'auto'. Explicitly pass `predict_method='predict' to replicate the old behavior, or pass `predict_method='auto' or other valid values to silence this warning.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/fairlearn/postprocessing/_threshold_optimizer.py:285: FutureWarning: 'predict_method' default value is changed fr

{
    'Logistic Regression': {
        0: {
            'y_test': array([0, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        1: {
            'y_test': array([0, 0, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 1, 1, 0]),
            's_test': array([1, 1, 0, ..., 1, 0, 1])
        },
        2: {
            'y_test': array([1, 1, 0, ..., 0, 0, 0]),
            'y_pred': array([1, 1, 1, ..., 0, 0, 0]),
            's_test': array([0, 1, 1, ..., 0, 1, 1])
        },
        3: {
            'y_test': array([0, 0, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 1]),
            's_test': array([0, 1, 1, ..., 0, 1, 0])
        },
        4: {
            'y_test': array([0, 0, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 1, 0, ..., 1, 0, 0]),
            's_test': array([1, 1, 0, ..., 0, 1, 1])
        },
        5: {
            'y_test': array([1, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 1, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        6: {
            'y_test': array([1, 1, 0, ..., 1, 1, 0]),
            'y_pred': array([0, 0, 1, ..., 1, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        7: {
            'y_test': array([0, 1, 0, ..., 1, 0, 1]),
            'y_pred': array([0, 1, 0, ..., 0, 1, 0]),
            's_test': array([0, 1, 1, ..., 0, 1, 1])
        },
        8: {
            'y_test': array([0, 0, 0, ..., 0, 0, 1]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 1]),
            's_test': array([0, 1, 1, ..., 1, 1, 1])
        },
        9: {
            'y_test': array([0, 1, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 1, 0, ..., 1, 1, 0]),
            's_test': array([1, 1, 1, ..., 0, 1, 1])
        }
    },
    'Decision Tree': {
        0: {
            'y_test': array([0, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 1, 1]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        1: {
            'y_test': array([0, 0, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 1, ..., 0, 1, 0]),
            's_test': array([1, 1, 0, ..., 1, 0, 1])
        },
        2: {
            'y_test': array([1, 1, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 1, 0, ..., 0, 1, 0]),
            's_test': array([0, 1, 1, ..., 0, 1, 1])
        },
        3: {
            'y_test': array([0, 0, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 1, 0, ..., 0, 1, 1]),
            's_test': array([0, 1, 1, ..., 0, 1, 0])
        },
        4: {
            'y_test': array([0, 0, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 1, 0, 0]),
            's_test': array([1, 1, 0, ..., 0, 1, 1])
        },
        5: {
            'y_test': array([1, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([1, 0, 0, ..., 0, 1, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        6: {
            'y_test': array([1, 1, 0, ..., 1, 1, 0]),
            'y_pred': array([0, 1, 1, ..., 1, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        7: {
            'y_test': array([0, 1, 0, ..., 1, 0, 1]),
            'y_pred': array([0, 1, 0, ..., 0, 1, 1]),
            's_test': array([0, 1, 1, ..., 0, 1, 1])
        },
        8: {
            'y_test': array([0, 0, 0, ..., 0, 0, 1]),
            'y_pred': array([0, 0, 0, ..., 1, 1, 0]),
            's_test': array([0, 1, 1, ..., 1, 1, 1])
        },
        9: {
            'y_test': array([0, 1, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 1, 1, ..., 1, 1, 0]),
            's_test': array([1, 1, 1, ..., 0, 1, 1])
        }
    },
    'Bagging': {
        0: {
            'y_test': array([0, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 1]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        1: {
            'y_test

# AIF360
[AIF360](https://aif360.mybluemix.net) is a system used in order to detect and mitigate a possible bias inside a dataset.
It requires firstly the identification of the corresponding protected attribute (sex, race, age etc.)

### Configuration

In order to use [AIF360](https://aif360.mybluemix.net) we need to define a Binary Dataset and set the privileged/unprivileged groups

In [ ]:
data_orig_aif = BinaryLabelDataset(
    favorable_label = 1,
    unfavorable_label = 0,
    df = df.copy(),
    label_names = [target_variable],
    protected_attribute_names = [sensible_attribute])

privileged_groups = [{'race_category': 1}]  #todo mettere sensible_attribute
unprivileged_groups = [{'race_category': 0}]

print(data_orig_aif.privileged_protected_attributes)

[array([1.])]

## Pre-processing

### Reweighing

Pre-processing: dataset is equal to the original one, the predictor is the same, only use different weights

NO PARAMETER

NO MITIGATED DATASET

NO MODEL ENFORCEMENT

In [ ]:
def aif360_rw(config):
  mitigation = 'aif360-rw'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  predictions_and_tests = {}
  for model_name in models:
    clf = models[model_name]
    RW = Reweighing(unprivileged_groups = unprivileged_groups,
                privileged_groups = privileged_groups)

    df_splitting = train_test_splitting(df, n_splits)
    pred_and_y = {}
    for i in range(0,n_splits):
      df_split = df_splitting[i]
      df_train = df_split['train']
      df_test = df_split['test']

      X_train, Y_train, X_test, Y_test = df_X_Y_split(df_train, df_test, target_variable)
      S_test = df_test[sensible_attribute].values

      data_orig_aif = BinaryLabelDataset(favorable_label = 1, unfavorable_label = 0, df = df_train.copy(), label_names = [target_variable], protected_attribute_names = [sensible_attribute])
      rw_dataset = RW.fit_transform(data_orig_aif)
      rw_df = rw_dataset.convert_to_dataframe()
      sample_weights= rw_df[1]["instance_weights"]


      clf.fit(X_train,Y_train,sample_weight=sample_weights)
      y_pred = clf.predict(X_test)
      pred_and_y[i] = {'y_test': Y_test.to_numpy().astype(int), 'y_pred': y_pred.astype(int), 's_test':  S_test.astype(int)}

      predictions_and_tests[model_name] = pred_and_y

  save_predictions_and_tests(predictions_and_tests, dataset_name, sensible_attribute, mitigation, path_to_project)

  return predictions_and_tests

In [ ]:
print(aif360_rw(config))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

### Disparate Impact Remover

Pre-processing: It transforms the dataset in a new mitigated one

PARAMETER REPAIR LEVEL

MITIGATED DATASET

NO MODEL ENFORCEMENT

In [ ]:
def aif360_di(config):
  mitigation = 'aif360-di'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  predictions_and_tests = {}

  #levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
  DIR = DisparateImpactRemover(repair_level=0.5)
  data_orig_aif = BinaryLabelDataset(favorable_label = 1, unfavorable_label = 0, df = df.copy(), label_names = [target_variable], protected_attribute_names = [sensible_attribute])

  train_repd = DIR.fit_transform(data_orig_aif)
  mit_aif360_di = train_repd.convert_to_dataframe()[0]

  save_mitigated_dataset(mit_aif360_di,path_to_project,dataset_name, sensible_attribute, mitigation)

  predictions_and_tests = compute_predictions_and_tests(mit_aif360_di, target_variable, n_splits, models, n_estimators, random_seed)

  save_predictions_and_tests(predictions_and_tests, dataset_name, sensible_attribute, mitigation, path_to_project)

  return predictions_and_tests

In [ ]:
print(aif360_di(config))

### Learning Fair Representation

PARAMETERS k, Ax, Ay, Az maxitem e maxfun

MITIGATED DATASET

NO MODEL ENFORCEMENT

In [ ]:
def aif360_lfr(config):
  mitigation = 'aif360-lfr'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  predictions_and_tests = {}

  TR = LFR(unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
    #seed= random_seed, k=10, Ax=0.01, Ay=1.0, Az=50.0,
    seed= random_seed, k=4, verbose=1)
  data_orig_aif = BinaryLabelDataset(favorable_label = 1, unfavorable_label = 0, df = df_reduced.copy(), label_names = [target_variable], protected_attribute_names = [sensible_attribute])

  TR = TR.fit(data_orig_aif, maxiter=500, maxfun=500) # Changed 5000 to 500
  transf_dataset = TR.transform(data_orig_aif)
  mit_aif360_lfr = transf_dataset.convert_to_dataframe()[0]

  save_mitigated_dataset(mit_aif360_lfr,path_to_project,dataset_name, sensible_attribute, mitigation)
  predictions_and_tests = compute_predictions_and_tests(mit_aif360_lfr, target_variable, n_splits, models, n_estimators, random_seed)
  save_predictions_and_tests(predictions_and_tests, dataset_name, sensible_attribute, mitigation, path_to_project)

  return predictions_and_tests

In [ ]:
predictions_and_tests = aif360_lfr(config)
print(predictions_and_tests)

In [ ]:
performance_metrics = compute_performance_metrics(predictions_and_tests, models, n_splits)
print(performance_metrics)

### Optimized Preprocessing

Pre-processing: mitigated dataset
Richiede valori binari come input quindi è stato fatto un pre-processing sui valori per essere 0,1

OptTool difficile da creare, chiesto a Daniel delucidazioni

PARAMETERS distortion_fun, epsilon, clist, dlist

MITIGATED DATASET

NO MODEL ENFORCEMENT

In [ ]:
def get_distortion(vold,vnew): # NEED TO CHANGE THIS
        bad_val=3
        OutNew=vnew["diabetes"]
        OutOld=vold["diabetes"]

        if not apply_dimensionality_reduction:
          InsOld=vold["num_lab_procedures"]
          InsNew=vnew["num_lab_procedures"]
        else:
          InsOld = 0
          InsNew = 0

        if ((OutNew>OutOld)& (InsNew<InsOld)): #| ((OutNew>OutOld) & (GluNew<GluOld)):
            return bad_val
        else:
            return 0

In [ ]:
def distortion_function(df):
  df2 = df.copy()
  df2['time_in_hospital'] = np.where(df2['time_in_hospital'].between(1,6), 1, 0) # 1 valore giusto, 0 anomalo
  df2["number_inpatient"]=np.where(df2["number_inpatient"] < 1, 1, 0)
  df2["number_diagnoses"]=np.where(df2["number_diagnoses"].between(1,10), 1, 0)

  if not apply_dimensionality_reduction:
    df2 = df2.drop(columns=['glipizide_category', 'glyburide_category']) # drop non-binary columns that are full of zero's
    df2["num_procedures"]=np.where(df2["num_procedures"] < 5, 1, 0)
    df2["num_medications"]=np.where(df2["num_medications"] < 20, 1, 0)
    df2['num_lab_procedures'] = np.where(df2['num_lab_procedures'] < 50, 1, 0) # based on distribution

  return df2

In [ ]:
def aif360_op(config):
  mitigation = 'aif360-op'
  df, target_variable, sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  if apply_dimensionality_reduction:
    df_op = distortion_function(df_reduced)
  else:
    df_op = distortion_function(df)

  data_orig_aif_op = BinaryLabelDataset(
    favorable_label = 1,
    unfavorable_label = 0,
    df = df_op.copy(),
    label_names = [target_variable],
    protected_attribute_names = [sensible_attribute],
    metadata = default_mappings)

  optim_options = {
    "distortion_fun": get_distortion,
    "epsilon": 0.1,
    "clist": [0.99, 1.99, 2.99],
    "dlist": [.1, 0.05, 0]
  }

  predictions_and_tests = {}
  OP = OptimPreproc(OptTools, optim_options)
  dataset_orig_train, dataset_orig_test = data_orig_aif_op.split([0.7], shuffle=True)

  OP_fitted= OP.fit(dataset_orig_train)
  transf_dataset = OP_fitted.transform(data_orig_aif_op, transform_Y=True)
  mit_aif360_op = transf_dataset.convert_to_dataframe()[0]

  save_mitigated_dataset(mit_aif360_op,path_to_project,dataset_name, sensible_attribute, mitigation)

  predictions_and_tests = compute_predictions_and_tests(mit_aif360_op, target_variable, n_splits, models, n_estimators, random_seed)

  save_predictions_and_tests(predictions_and_tests, dataset_name, sensible_attribute, mitigation, path_to_project)

  return predictions_and_tests

In [ ]:
print(aif360_op(config))

##In-processing

Ogni tecnica genera un nuovo classificatore: no score and no mitigated dataset

### Adversarial Debiasing
In-processing: dataset is equal to the original one, the predictor is fairer by construction and given

PARAMETER SESSION

NO MITIGATED DATASET

NO SCORE FUNCTION

MODEL ENFORCEMENT

In [ ]:
def aif360_ad(config):
  mitigation = 'aif360-ad'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  predictions_and_tests = {}

  df_splitting = train_test_splitting(df, n_splits)
  pred_and_y = {}
  for i in range(0,n_splits):
    # Create a new session for each fold
    sess = tf.Session()
    # Added a unique scope name using the fold index
    AD = AdversarialDebiasing(privileged_groups = privileged_groups,
                        unprivileged_groups = unprivileged_groups,
                        scope_name=f'plain_classifier_{i}', # Changed scope name
                        debias=False,
                        sess=sess)

    df_split = df_splitting[i]
    df_train = df_split['train']
    df_test = df_split['test']

    X_train, Y_train, X_test, Y_test, S_train, S_test = df_X_Y_split_S(df_train, df_test, target_variable, sensible_attribute)

    data_orig_aif_train = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_train.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

    data_orig_aif_test = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_test.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

    AD.fit(data_orig_aif_train)

    predictions = AD.predict(data_orig_aif_test)

    mit_aif360_ad = predictions.convert_to_dataframe()[0]
    y_pred = mit_aif360_ad[target_variable]
    pred_and_y[i] = {'y_test': Y_test.to_numpy(), 'y_pred': y_pred.to_numpy().astype(int), 's_test':  S_test}

    predicted_and_real_tests = pred_and_y

    # Close the session after each fold
    sess.close()
  print(predicted_and_real_tests)
  save_predictions_and_tests(predicted_and_real_tests, dataset_name, sensible_attribute, mitigation, path_to_project)
  return predicted_and_real_tests

In [ ]:
aif360_ad(config)

### Prejudice remover
In-processing: dataset is equal to the original one, the predictor is fairer by construction and given

NON FUNZIONA!

PARAMETER eta

NO MITIGATED DATASET

NO SCORE FUNCTION

MODEL ENFORCEMENT



In [ ]:
def aif360_pr(config):
  mitigation = 'aif360-pr'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  predictions_and_tests = {}

  df_splitting = train_test_splitting(df, n_splits)
  pred_and_y = {}
  for i in range(0,n_splits):

    PR = PrejudiceRemover(eta=1.0, sensitive_attr=sensible_attribute, class_attr=target_variable)

    df_split = df_splitting[i]
    df_train = df_split['train']
    df_test = df_split['test']

    X_train, Y_train, X_test, Y_test, S_train, S_test = df_X_Y_split_S(df_train, df_test, target_variable, sensible_attribute)

    data_orig_aif_train = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_train.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

    data_orig_aif_test = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_test.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

    PR.fit(data_orig_aif_train)
    predictions = PR.predict(data_orig_aif_test)
    mit_aif360_pr = predictions.convert_to_dataframe()[0]

    y_pred = mit_aif360_pr[target_variable]
    pred_and_y[i] = {'y_test': Y_test.to_numpy(), 'y_pred': y_pred.to_numpy().astype(int), 's_test':  S_test}

    predicted_and_real_tests = pred_and_y
  print(predicted_and_real_tests)
  save_predictions_and_tests(predicted_and_real_tests, dataset_name, sensible_attribute, mitigation, path_to_project)
  return predicted_and_real_tests

In [ ]:
aif360_pr(config)

### Exponentiated Gradient Reduction
In-processing: dataset is equal to the original one, the predictor is fairer by construction and given

PARAMETERS constraint e estimator

NO MITIGATED DATASET

NO SCORE FUNCTION

MODEL ENFORCEMENT

In [ ]:
def aif360_er(config):
  mitigation = 'aif360-er'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)
  df_splitting = train_test_splitting(df, n_splits)
  pred_and_y = {}
  predictions_and_tests = {}
  for i in range(0,n_splits):

    df_split = df_splitting[i]
    df_train = df_split['train']
    df_test = df_split['test']

    X_train, Y_train, X_test, Y_test, S_train, S_test = df_X_Y_split_S(df_train, df_test, target_variable, sensible_attribute)

    data_orig_aif_train = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_train.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

    data_orig_aif_test = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_test.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

    lmod = DecisionTreeClassifier(max_depth=50)
    #lmod = LogisticRegression(solver='newton-cholesky', max_iter=500) # Not converging, is this normal?? #liblinear, sag, newton-cholesky

    exp_grad_red = ExponentiatedGradientReduction(estimator=lmod, constraints='DemographicParity', drop_prot_attr=False)
    exp_grad_red.fit(data_orig_aif_train)
    predictions = exp_grad_red.predict(data_orig_aif_test)

    mit_aif360_er = predictions.convert_to_dataframe()[0]
    y_pred = mit_aif360_er[target_variable]
    pred_and_y[i] = {'y_test': Y_test.to_numpy(), 'y_pred': y_pred.to_numpy().astype(int), 's_test':  S_test}

  predictions_and_tests = pred_and_y

  print(predictions_and_tests)
  save_predictions_and_tests(predictions_and_tests, dataset_name, sensible_attribute, mitigation, path_to_project)
  return predictions_and_tests


In [ ]:
aif360_er(config)

## Post-processing

In the first phase there is the model building: the same for Calibrated Equalized Post-processing, Equalize Odds Post_processing and Reject Option Classification


In [ ]:
def compute_predictions(x_train, y_train, x_test, model):
  model.fit(x_train, y_train)
  return model.predict(x_test)

### Calibrated Equalized Post-Processing

Post-processing: model's labels are changes, mitigated dataset is equal to the original one

PARAMETER cost_constraint

NO MITIGATED DATASET

NO EMBEDDED SCORE FUNCTION

NO MODEL ENFORCEMENT

In [ ]:
def aif360_ce(config):
  mitigation = 'aif360-ce'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  predictions_and_tests = {}
  for model_name in models:
    clf = models[model_name]
    CEPP = CalibratedEqOddsPostprocessing(unprivileged_groups= unprivileged_groups, privileged_groups = privileged_groups, cost_constraint='weighted', seed=random_seed)

    df_splitting = train_test_splitting(df, n_splits)
    pred_and_y = {}
    for i in range(0,n_splits):
      df_split = df_splitting[i]
      df_train = df_split['train']
      df_test = df_split['test']

      X_train, Y_train, X_test, Y_test = df_X_Y_split(df_train,
                                                      df_test,
                                                      target_variable)
      S_test = df_test[sensible_attribute].values

      # Compute the prediction for this model
      Y_pred = compute_predictions(X_train, Y_train, X_test, clf)

      # Define a new data frame with the predicted values as labels
      df_pred = df.loc[Y_test.index]
      df_pred[target_variable] = Y_pred


      # Create the aif datasets with y_test and y_pred data frames
      data_orig_aif_test = BinaryLabelDataset(favorable_label = 1,
                                              unfavorable_label = 0,
                                              df = df.iloc[Y_test.index],
                                              label_names = [target_variable],
                                              protected_attribute_names =
                                               [sensible_attribute])

      data_orig_aif_pred = BinaryLabelDataset(favorable_label = 1,
                                              unfavorable_label = 0,
                                              df = df_pred.copy(),
                                              label_names = [target_variable],
                                              protected_attribute_names =
                                               [sensible_attribute])


      # Fit the post-processing technique with these two datasets i.e. test, pred
      new_predictions = CEPP.fit_predict(data_orig_aif_test, data_orig_aif_pred)
      mit_aif360_cpp = new_predictions.convert_to_dataframe()[0]
      y_pred = mit_aif360_cpp[target_variable]

      '''score = accuracy_score(Y_test, y_pred)
      print("score ", score, '\n')'''

      pred_and_y[i] = {'y_test': Y_test.to_numpy(), 'y_pred': y_pred.to_numpy().astype(int), 's_test':  S_test}

      predictions_and_tests[model_name] = pred_and_y

  print(predictions_and_tests)

  save_predictions_and_tests(predictions_and_tests, dataset_name, sensible_attribute, mitigation, path_to_project)
  return predictions_and_tests

In [ ]:
aif360_ce(config)

### Equalize Odds Post-Processing
Post-processing: model's labels are changes, mitigated dataset is equal to the original one

NO PARAMETER

NO MITIGATED DATASET

NO EMBEDDED SCORE FUNCTION

NO MODEL ENFORCEMENT

In [ ]:
def aif360_eo(config):
  mitigation = 'aif360-eo'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  predictions_and_tests = {}
  for model_name in models:
    clf = models[model_name]
    EOPP = EqOddsPostprocessing(unprivileged_groups= unprivileged_groups,
                                privileged_groups = privileged_groups,
                                seed=random_seed)

    df_splitting = train_test_splitting(df, n_splits)
    pred_and_y = {}
    for i in range(0,n_splits):
      df_split = df_splitting[i]
      df_train = df_split['train']
      df_test = df_split['test']

      X_train, Y_train, X_test, Y_test = df_X_Y_split(df_train,
                                                      df_test,
                                                      target_variable)
      S_test = df_test[sensible_attribute].values

      # Compute the prediction for this model
      Y_pred = compute_predictions(X_train, Y_train, X_test, clf)

      # Define a new data frame with the predicted values as labels
      df_pred = df.loc[Y_test.index]
      df_pred[target_variable] = Y_pred


      # Create the aif datasets with y_test and y_pred data frames
      data_orig_aif_test = BinaryLabelDataset(favorable_label = 1,
                                              unfavorable_label = 0,
                                              df = df.iloc[Y_test.index],
                                              label_names = [target_variable],
                                              protected_attribute_names =
                                               [sensible_attribute])

      data_orig_aif_pred = BinaryLabelDataset(favorable_label = 1,
                                              unfavorable_label = 0,
                                              df = df_pred.copy(),
                                              label_names = [target_variable],
                                              protected_attribute_names =
                                               [sensible_attribute])


      # Fit the post-processing technique with these two datasets i.e. test, pred
      new_predictions = EOPP.fit_predict(data_orig_aif_test, data_orig_aif_pred)
      mit_aif360_eopp = new_predictions.convert_to_dataframe()[0]
      y_pred = mit_aif360_eopp[target_variable]

      '''score = accuracy_score(Y_test, y_pred)
      print("score ", score, '\n')'''

      pred_and_y[i] = {'y_test': Y_test.to_numpy(),
                       'y_pred': y_pred.to_numpy().astype(int),
                       's_test':  S_test}

      predictions_and_tests[model_name] = pred_and_y

  print(predictions_and_tests)

  save_predictions_and_tests(predictions_and_tests, dataset_name, sensible_attribute, mitigation, path_to_project)

  return predictions_and_tests

In [ ]:
aif360_eo(config)

### Reject Option Classification

Post-processing: model's labels are changes, mitigated dataset is equal to the original one

NO MITIGATED DATASET

PARAMETERS: low_class_thresh,high_class_thresh, num_class_thresh, num_ROC_margin, metric_name, metric_ub, metric_lb

NO EMBEDDED SCORE FUCTION

NO MODEL ENFORCEMENT

In [ ]:
def aif360_roc(config):
  mitigation = 'aif360-roc'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  if apply_dimensionality_reduction:
    df = df_reduced

  predictions_and_tests = {}
  for model_name in tqdm(models):
    clf = models[model_name]
    ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups,
                                     privileged_groups=privileged_groups,
                                     low_class_thresh=0.01, high_class_thresh=0.99,
                                     num_class_thresh=100, num_ROC_margin=50,
                                     metric_name="Equal opportunity difference",
                                     metric_ub=0.05, metric_lb=-0.05)

    df_splitting = train_test_splitting(df, n_splits)
    pred_and_y = {}
    for i in range(0,n_splits):
      df_split = df_splitting[i]
      df_train = df_split['train']
      df_test = df_split['test']

      X_train, Y_train, X_test, Y_test = df_X_Y_split(df_train,
                                                      df_test,
                                                      target_variable)
      S_test = df_test[sensible_attribute].values

      # Compute the prediction for this model
      Y_pred = compute_predictions(X_train, Y_train, X_test, clf)

      # Define a new data frame with the predicted values as labels
      df_pred = df.loc[Y_test.index]
      df_pred[target_variable] = Y_pred

      # Create the aif datasets with y_test and y_pred data frames
      data_orig_aif_test = BinaryLabelDataset(favorable_label = 1,
                                              unfavorable_label = 0,
                                              df = df.iloc[Y_test.index],
                                              label_names = [target_variable],
                                              protected_attribute_names =
                                               [sensible_attribute])

      data_orig_aif_pred = BinaryLabelDataset(favorable_label = 1,
                                              unfavorable_label = 0,
                                              df = df_pred.copy(),
                                              label_names = [target_variable],
                                              protected_attribute_names =
                                               [sensible_attribute])

      # Fit the post-processing technique with these two datasets i.e. test, pred
      new_predictions = ROC.fit_predict(data_orig_aif_test, data_orig_aif_pred)
      mit_aif360_eopp = new_predictions.convert_to_dataframe()[0]
      y_pred = mit_aif360_eopp[target_variable]

      '''score = accuracy_score(Y_test, y_pred)
      print("score ", score, '\n')'''

      pred_and_y[i] = {'y_test': Y_test.to_numpy(), 'y_pred': y_pred.to_numpy().astype(int), 's_test':  S_test}

      predictions_and_tests[model_name] = pred_and_y

  print(predictions_and_tests)

  save_predictions_and_tests(predictions_and_tests, dataset_name, sensible_attribute, mitigation, path_to_project)

  return predictions_and_tests

In [ ]:
aif360_roc(config)